# Installations

In [96]:
# Packages should be pre-installed in the environment
# Dependencies: langgraph langchain-openai langchain-core langchain-google-genai pyyaml
pass

# Imports

In [97]:
from typing import TypedDict, Dict, List, Annotated
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langchain_core.messages import BaseMessage, HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_google_genai import ChatGoogleGenerativeAI

In [98]:
import os
import yaml
from IPython.display import Image, display
from datetime import datetime
import yaml
import re
import tempfile
from typing import Dict

# API key

In [99]:
# # Read API key from environment variable or file
# from dotenv import load_dotenv
# # First try to get API key from environment variable (for Cloud Run)
# load_dotenv()
api_key = os.environ.get('GOOGLE_API_KEY')

# print(api_key)
api_key_source = "environment variable"

if api_key:
    print(f"✅ API key loaded from environment variable")
else:
    # Fallback to file-based approach for local development
    # Get current working directory and script directory
    current_dir = os.getcwd()
    script_dir = os.path.dirname(os.path.abspath(__file__)) if '__file__' in globals() else current_dir

    # Try to find the api_key file in multiple possible locations
    possible_paths = [
        'api_key',  # Current directory
        os.path.join(script_dir, 'api_key'),  # Same directory as script/notebook
        os.path.join(current_dir, '..', 'api_key'),  # Parent directory (flask-app level)
        os.path.join(script_dir, '..', 'api_key'),  # Parent of script directory
        '/Users/leo/Documents/uni/master/25sose/NLP-social/WEBDEMO/flask-app/api_key',  # Flask app directory
        '/Users/leo/Documents/uni/master/25sose/NLP-social/WEBDEMO/flask-app/backend/api_key'  # Backend directory
    ]

    api_key_source = None

    for path in possible_paths:
        try:
            if os.path.exists(path):
                with open(path, 'r') as f:
                    api_key = f.read().strip()
                api_key_source = path
                print(f"✅ API key loaded from file: {path}")
                break
        except Exception as e:
            continue

    if api_key is None:
        print("❌ API key not found in environment variable or any file location.")
        print(f"   Current working directory: {current_dir}")
        print("   For Cloud Run: Set GOOGLE_API_KEY environment variable")
        print("   For local development: Create an 'api_key' file with your Google API key")
        print("   Searched file paths:")
        for path in possible_paths:
            abs_path = os.path.abspath(path)
            exists = "✓" if os.path.exists(path) else "✗"
            print(f"   {exists} {path} -> {abs_path}")
        print("   Get your API key from: https://aistudio.google.com/app/apikey")

# Uncomment and use these if you prefer environment variables instead:
# os.environ["OPENAI_API_KEY"] = ""
# os.environ["GOOGLE_API_KEY"] = ""

✅ API key loaded from environment variable


# LLM

## OpenAI

In [100]:
# # Initialize LLM
# llm = ChatOpenAI(model="gpt-4", temperature=0.7)

## Gemini

https://ai.google.dev/gemini-api/docs/rate-limits

In [101]:
# # Initialize Gemini LLM (using free model)
# llm = ChatGoogleGenerativeAI(
#     model="gemini-2.5-flash-preview-05-20",  # Free model
#     temperature=0.7,
#     google_api_key=api_key
# )

In [102]:
# Initialize Gemini LLM (using free model)
if api_key:
    llm = ChatGoogleGenerativeAI(
        model="gemini-2.0-flash-001",  # Free model
        temperature=0.3,
        google_api_key=api_key
    )
    print("✅ LLM gemini-2.0-flash-001 initialized successfully")
else:
    llm = None
    print("❌ Cannot initialize LLM - no API key available")

✅ LLM gemini-2.0-flash-001 initialized successfully


## Test API connection

In [103]:
# Test API connection
if llm is not None:
    try:
        print("🔍 Testing API connection...")
        test_response = llm.invoke([{"role": "user", "content": "Hello, can you respond with just 'API connection successful'?"}])
        print(f"✅ API test successful: {test_response.content}")
    except Exception as e:
        print(f"❌ API connection failed: {e}")
        print("\nPossible solutions:")
        print("1. Check your internet connection")
        print("2. Verify your API key is correct and active")
        print("3. Check if you have sufficient API quota")
        print("4. Try again in a few moments (rate limiting)")
else:
    print("⚠️  Cannot test API - LLM not initialized")

🔍 Testing API connection...
✅ API test successful: API connection successful


## Troubleshooting Connection Issues

If you're experiencing connection errors, try these solutions:

### Common Issues:
1. **Invalid API Key**: Make sure your API key is correct and active
2. **Network Issues**: Check your internet connection
3. **Rate Limiting**: Google API has rate limits - wait a few minutes and try again
4. **Quota Exceeded**: Check your API usage quota at [Google AI Studio](https://aistudio.google.com/)
5. **Firewall/Proxy**: Corporate networks might block API calls

### Quick Fixes:
- Restart the notebook kernel and try again
- Verify the `api_key` file contains only your API key (no extra spaces/characters)
- Try running the API test cell above to isolate the issue
- Check [Google AI Studio](https://aistudio.google.com/app/apikey) to verify your key is working

# Cases

In [104]:
def load_case_from_file(file_path: str, scenario_number: int = None) -> str:
    """Load case details from text file

    Args:
        file_path: Path to the case file
        scenario_number: If file contains multiple scenarios, specify which one (1, 2, 3, etc.)

    Returns:
        Case details as string
    """
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read().strip()

        # Check if file contains multiple scenarios
        if 'Scenario 1:' in content and 'Scenario 2:' in content:
            scenarios = {}

            # Split by scenario markers
            parts = content.split('Scenario ')
            for part in parts[1:]:  # Skip first empty part
                if ':' in part:
                    scenario_num = int(part.split(':')[0].strip())
                    scenario_text = 'Scenario ' + part
                    scenarios[scenario_num] = scenario_text.strip()

            if scenario_number:
                if scenario_number in scenarios:
                    return scenarios[scenario_number]
                else:
                    available = list(scenarios.keys())
                    raise ValueError(f"Scenario {scenario_number} not found. Available scenarios: {available}")
            else:
                # Return all scenarios combined
                return content
        else:
            # Single case file
            return content

    except FileNotFoundError:
        raise FileNotFoundError(f"Case file {file_path} not found")
    except Exception as e:
        raise Exception(f"Error loading case from {file_path}: {e}")

In [105]:
def select_scenario(scenarios: Dict[str, str]) -> str:
    """Allow user to select a scenario from loaded cases"""
    if not scenarios:
        return None

    print("\nAvailable scenarios:")
    scenario_list = list(scenarios.keys())

    for i, scenario in enumerate(scenario_list, 1):
        # Extract just the scenario title for display
        title = scenario
        if ':' in scenarios[scenario]:
            first_line = scenarios[scenario].split('\n')[0]
            if 'Background:' in first_line:
                title = f"{scenario} - Murder case with eyewitness evidence"
        print(f"{i}. {title}")

    while True:
        try:
            choice = input(f"\nSelect scenario (1-{len(scenario_list)}) or 'back': ")
            if choice.lower() == 'back':
                return None

            choice_num = int(choice)
            if 1 <= choice_num <= len(scenario_list):
                selected_scenario = scenario_list[choice_num - 1]
                case_details = scenarios[selected_scenario]

                # Display selected case
                print(f"\n=== SELECTED CASE ===")
                print(case_details[:500] + "..." if len(case_details) > 500 else case_details)
                print("=" * 50)

                confirm = input("\nUse this scenario? (y/n): ")
                if confirm.lower() in ['y', 'yes']:
                    return case_details
            else:
                print(f"Please enter a number between 1 and {len(scenario_list)}")
        except ValueError:
            print("Please enter a valid number")

In [106]:
def initialize_with_case(case_file_path: str, scenario_number: int = None):
    """Initialize the system with a case from file"""
    global current_case, current_case_filename, current_scenario_number

    try:
        current_case = load_case_from_file(case_file_path, scenario_number)
        current_case_filename = case_file_path  # Track the filename
        current_scenario_number = scenario_number  # Track the scenario number

        if scenario_number:
            print(f"✅ Loaded Scenario {scenario_number} from {case_file_path}")
        else:
            print(f"✅ Loaded case from {case_file_path}")

        # Show preview of the case
        preview = current_case[:200] + "..." if len(current_case) > 200 else current_case
        print(f"Case Preview: {preview}\n")

    except Exception as e:
        print(f"❌ Error loading case: {e}")
        raise

In [107]:
def list_scenarios_in_file(file_path: str):
    """List available scenarios in a case file"""
    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        scenarios = []
        parts = content.split('Scenario ')
        for part in parts[1:]:
            if ':' in part:
                scenario_num = int(part.split(':')[0].strip())
                # Get first line after the colon as title
                title_line = part.split('\n')[0] if '\n' in part else part[:50]
                scenarios.append((scenario_num, title_line))

        return scenarios
    except Exception as e:
        print(f"Error reading file {file_path}: {e}")
        return []


# Jury

In [108]:
class JuryState(TypedDict):
    messages: Annotated[List[BaseMessage], add_messages]
    case_details: str
    jury_backgrounds: Dict[str, str]
    current_round: int
    current_juror_index: int  # Track which juror is speaking within the round
    total_rounds: int  # Total number of deliberation rounds
    jury_order: List[str]  # Order of jury members

In [109]:
# Jury members and their backgrounds - FALLBACK
JURY_MEMBERS = {
    "Alice": "Retired teacher, 30 years experience. Values fairness and second chances.",
    "Bob": "Small business owner. Practical, fact-focused, believes in personal responsibility.",
    "Carol": "Social worker with family court experience. Empathetic, considers circumstances.",
    "David": "Engineer with technical background. Data-driven, prefers clear evidence."
}

## Files

In [110]:
def load_backgrounds_from_files(file_paths: List[str]) -> Dict[str, str]:
    """Load jury backgrounds from text files (legacy function for backward compatibility)"""
    backgrounds = {}
    jury_names = list(JURY_MEMBERS.keys())

    for i, file_path in enumerate(file_paths):
        if i >= len(jury_names):
            break
        try:
            with open(file_path, 'r') as f:
                backgrounds[jury_names[i]] = f.read().strip()
        except FileNotFoundError:
            print(f"File {file_path} not found, using default background")
            backgrounds[jury_names[i]] = JURY_MEMBERS[jury_names[i]]

    # Fill remaining with defaults
    for name in jury_names[len(file_paths):]:
        backgrounds[name] = JURY_MEMBERS[name]

    return backgrounds

## YAML

In [111]:
def load_backgrounds_from_yaml(file_path: str) -> Dict[str, str]:
    """Load jury backgrounds from YAML file supporting multiple structures"""
    backgrounds = {}

    try:
        with open(file_path, 'r') as f:
            data = yaml.safe_load(f)

        for jury_key, jury_data in data.items():
            # Detect structure type by checking for key fields
            if 'first_name' in jury_data and 'last_name' in jury_data:
                # Detailed structure (like jurors.yaml)
                background = _process_detailed_structure(jury_data)
                full_name = f"{jury_data.get('first_name', 'Unknown')} {jury_data.get('last_name', 'Unknown')}"

            elif 'backstory' in jury_data:
                # Simplified structure (like agents.yaml, old_and_young.yaml)
                background = _process_simplified_structure(jury_data)
                full_name = _extract_name_from_backstory(jury_data.get('backstory', ''), jury_key)

            else:
                # Unknown structure - use available data
                background = _process_unknown_structure(jury_data)
                full_name = jury_key.replace('_', ' ').title()

            backgrounds[full_name] = background

    except FileNotFoundError:
        print(f"YAML file {file_path} not found, using default backgrounds")
        return JURY_MEMBERS.copy() if 'JURY_MEMBERS' in globals() else {}
    except yaml.YAMLError as e:
        print(f"Error parsing YAML file {file_path}: {e}")
        return JURY_MEMBERS.copy() if 'JURY_MEMBERS' in globals() else {}
    except Exception as e:
        print(f"Error loading backgrounds from {file_path}: {e}")
        return JURY_MEMBERS.copy() if 'JURY_MEMBERS' in globals() else {}

    return backgrounds


def _process_detailed_structure(jury_data: Dict) -> str:
    """Process detailed structure with separate fields for personal information"""
    biography = jury_data.get('biography', '')
    age = jury_data.get('age', 'Unknown age')
    education = jury_data.get('education', 'Unknown education')
    occupation = jury_data.get('occupation', 'Unknown occupation')
    income = jury_data.get('income', 'Unknown income')
    state = jury_data.get('state', 'Unknown state')
    religion = jury_data.get('religion', 'Unknown religion')
    race = jury_data.get('race', 'Unknown race')
    gender = jury_data.get('gender', 'Unknown gender')
    goal = jury_data.get('goal', 'Serve justice fairly')
    role = jury_data.get('role', 'Regular juror')

    # Combine all information into a comprehensive background
    background = f"{biography}\n\n"
    background += f"Personal Details: {age}, {gender}, {race}, {education}, {occupation} from {state}. "
    background += f"Income: {income}. Religion: {religion}.\n"
    background += f"Role: {role}\n"
    background += f"Goal: {goal}"

    return background


def _process_simplified_structure(jury_data: Dict) -> str:
    """Process simplified structure with backstory, role, and goal"""
    backstory = jury_data.get('backstory', '').strip()
    role = jury_data.get('role', '').strip()
    goal = jury_data.get('goal', '').strip()

    # Clean up multiline strings and remove template placeholders
    backstory = re.sub(r'\s+', ' ', backstory)
    role = re.sub(r'\s+', ' ', role)
    goal = re.sub(r'\s+', ' ', goal)

    # Remove template placeholders like {topic} and {current_year}
    backstory = re.sub(r'\{[^}]+\}', '[case topic]', backstory)
    role = re.sub(r'\{[^}]+\}', '[case topic]', role)
    goal = re.sub(r'\{[^}]+\}', '[case topic]', goal)

    background = f"{backstory}\n\n"
    if role and role != backstory:
        background += f"Role: {role}\n"
    if goal and goal != backstory:
        background += f"Goal: {goal}"

    return background.strip()


def _process_unknown_structure(jury_data: Dict) -> str:
    """Process unknown structure by using all available string data"""
    background_parts = []

    for key, value in jury_data.items():
        if isinstance(value, str) and value.strip():
            # Clean up multiline strings
            clean_value = re.sub(r'\s+', ' ', value.strip())
            background_parts.append(f"{key.replace('_', ' ').title()}: {clean_value}")

    return "\n".join(background_parts) if background_parts else "No background information available."


def _extract_name_from_backstory(backstory: str, fallback_key: str) -> str:
    """Extract a name from the backstory text, with fallback to jury key"""
    if not backstory:
        return fallback_key.replace('_', ' ').title()

    # Look for name patterns in the backstory
    # Pattern 1: "Name is a..." or "Name, a..."
    name_pattern1 = r'^([A-Z][a-z]+ [A-Z][a-z]+)\s+(?:is|,)'
    match1 = re.search(name_pattern1, backstory)
    if match1:
        return match1.group(1)

    # Pattern 2: Just first sentence that might contain a name
    name_pattern2 = r'^([A-Z][a-z]+ [A-Z][a-z]+)'
    match2 = re.search(name_pattern2, backstory)
    if match2:
        return match2.group(1)

    # Pattern 3: Look for any capitalized name in the first 50 characters
    name_pattern3 = r'([A-Z][a-z]+ [A-Z][a-z]+)'
    match3 = re.search(name_pattern3, backstory[:50])
    if match3:
        return match3.group(1)

    # Fallback to jury key
    return fallback_key.replace('_', ' ').title()


In [112]:
def initialize_with_yaml(yaml_file_path: str, total_rounds: int = 3):
    """Initialize the graph with jury data from YAML file"""
    global graph, jury_backgrounds, current_total_rounds, current_jury_filename

    if llm is None:
        print("Cannot initialize - API key not configured")
        return

    graph, jury_backgrounds, current_total_rounds = create_jury_graph(yaml_file=yaml_file_path, total_rounds=total_rounds)
    current_total_rounds = total_rounds
    current_jury_filename = yaml_file_path  # Track the filename

    print(f"Loaded jury members from {yaml_file_path}:")
    for name in jury_backgrounds.keys():
        print(f"  - {name}")
    print(f"Set to {total_rounds} deliberation rounds")
    print()


## Create jury node

### Rate Limiting Configuration

Google Gemini API has these free tier limits:
- **15 requests per minute** for gemini-2.0-flash
- **1,500 requests per day** for free tier

The notebook includes automatic rate limiting and retry logic to handle quota exceeded errors.

In [113]:
def create_jury_node(jury_name: str):
    """Create a jury member node function"""
    import time
    import random
    
    def jury_response(state: JuryState):
        if llm is None:
            message = AIMessage(content="Cannot generate response - API key not configured", name=jury_name)
            return {"messages": [message]}

        background = state["jury_backgrounds"][jury_name]
        case = state["case_details"]
        current_round = state.get("current_round", 1)
        current_juror_index = state.get("current_juror_index", 0)

        # # Get recent conversation
        # recent_msgs = state["messages"][-6:] if len(state["messages"]) > 6 else state["messages"]

        # # Get messages from the last two rounds
        # recent_msgs = get_last_two_rounds_context(state["messages"], current_round)

        # context = "\n".join([f"{getattr(msg, 'name', 'User')}: {msg.content}" for msg in recent_msgs])

        # Get FULL conversation history
        all_messages = state["messages"]
        deliberation_msgs = []
        for msg in all_messages:
            # Skip the initial case presentation (HumanMessage)
            if isinstance(msg, HumanMessage):
                continue
            # For AIMessages, check if they have a name attribute
            if isinstance(msg, AIMessage) and hasattr(msg, 'name'):
                # Skip "=== X ===" style moderator messages but include other moderator messages
                if msg.name == "Moderator" and "===" in msg.content:
                    continue
                # Include all other messages (juror deliberations and non-procedural moderator messages)
                deliberation_msgs.append(f"{msg.name}: {msg.content}")
        context = "\n".join(deliberation_msgs)

        prompt = f"""You are {jury_name}, a jury member in Round {current_round} of deliberation.
Background: {background}

Case: {case}

Full deliberation so far:
{context}

As {jury_name}, give your perspective on this case. Consider what others have said and build on the discussion. Keep it to 2-3 sentences and be conversational.

At the end of your response, indicate your current stance by adding:
[Current stance: GUILTY/NOT GUILTY] - [brief reason referencing the deliberation]"""

        # Add rate limiting and retry logic
        max_retries = 3
        base_delay = 5  # seconds
        
        for attempt in range(max_retries):
            try:
                # Add a small delay between requests to avoid rate limiting
                if attempt > 0:
                    delay = base_delay * (2 ** attempt) + random.uniform(0, 2)
                    print(f"⏳ Rate limit hit for {jury_name}, waiting {delay:.1f} seconds...")
                    time.sleep(delay)
                else:
                    # Small delay even on first attempt
                    time.sleep(random.uniform(1, 3))
                
                response = llm.invoke([HumanMessage(content=prompt)])
                message = AIMessage(content=response.content, name=jury_name)
                break
                
            except Exception as e:
                error_msg = str(e)
                if "429" in error_msg or "quota" in error_msg.lower():
                    if attempt < max_retries - 1:
                        continue  # Retry with longer delay
                    else:
                        print(f"❌ Rate limit exceeded for {jury_name} after {max_retries} attempts")
                        message = AIMessage(content=f"I need more time to consider this case due to system limitations.", name=jury_name)
                else:
                    print(f"Error generating response for {jury_name}: {e}")
                    message = AIMessage(content=f"I need more time to consider this case.", name=jury_name)
                break

        # Advance to next juror
        next_juror_index = current_juror_index + 1

        return {
            "messages": [message],
            "current_juror_index": next_juror_index
        }

    return jury_response


In [114]:
def get_last_two_rounds_context(messages: List[BaseMessage], current_round: int) -> List[BaseMessage]:
    """Extract messages from the last two rounds of deliberation"""

    # Early return for initial rounds
    if current_round <= 2 or len(messages) <= 6:  # Fallback to at least 6 messages
        return messages

    # Track round transitions
    round_transitions = {}

    for i, msg in enumerate(messages):
        if hasattr(msg, 'name') and msg.name == "Moderator":
            if "=== DELIBERATION ROUND" in msg.content:
                try:
                    # Extract round number
                    round_str = msg.content.split("ROUND ")[1].split(" ===")[0]
                    round_num = int(round_str)
                    round_transitions[round_num] = i
                except (IndexError, ValueError):
                    continue

    # Determine starting index
    if current_round == 1:
        # Include all messages
        start_idx = 0
    elif current_round == 2:
        # Include from round 1 onwards
        start_idx = round_transitions.get(1, 0)
    else:
        # Include from two rounds ago
        two_rounds_ago = current_round - 1
        start_idx = round_transitions.get(two_rounds_ago, 0)

        # Fallback: if we can't find the round marker, use last N messages
        if start_idx == 0 and len(messages) > 12:
            # Estimate ~4-6 messages per round, so ~12 for 2 rounds
            start_idx = len(messages) - 12

    return messages[start_idx:]

# Moderator

In [115]:
def moderator(state: JuryState):
    """Enhanced moderator to manage multi-round deliberations"""
    current_round = state.get("current_round", 0)
    current_juror_index = state.get("current_juror_index", 0)
    total_rounds = state.get("total_rounds", 3)

    # Only announce the very beginning of deliberation
    if current_round == 0 and current_juror_index == 0:
        msg = AIMessage(content="=== JURY DELIBERATION BEGINS ===", name="Moderator")
        return {
            "messages": [msg],
            "current_round": current_round,
            "current_juror_index": current_juror_index
        }

    # For all other cases, just pass through without messages
    return {
        "current_round": current_round,
        "current_juror_index": current_juror_index
    }

# Final Verdict

In [116]:
def final_verdict(state: JuryState):
    """Collect final verdicts from all jury members"""
    import time
    import random
    
    if llm is None:
        summary = "=== FINAL VERDICTS ===\nCannot collect verdicts - API key not configured"
        return {"messages": [AIMessage(content=summary, name="Final_Verdict")]}

    case = state["case_details"]

    # # Get summary of key discussion points
    # discussion_summary = get_deliberation_summary(state["messages"])

    # Get FULL conversation history
    all_messages = state["messages"]
    deliberation_msgs = []
    for msg in all_messages:
        # Skip the initial case presentation (HumanMessage)
        if isinstance(msg, HumanMessage):
            continue
        # For AIMessages, check if they have a name attribute
        if isinstance(msg, AIMessage) and hasattr(msg, 'name'):
            # Skip "=== X ===" style moderator messages but include other moderator messages
            if msg.name == "Moderator" and "===" in msg.content:
                continue
            # Include all other messages (juror deliberations and non-procedural moderator messages)
            deliberation_msgs.append(f"{msg.name}: {msg.content}")
    context = "\n".join(deliberation_msgs)
    # print(context)

    # Get all jury members' final decisions
    verdicts = {}
    for i, jury_name in enumerate(state["jury_backgrounds"].keys()):
        background = state["jury_backgrounds"][jury_name]

        prompt = f"""You are {jury_name}, a jury member.
Background: {background}

Case: {case}

Full deliberation transcript:
{context}

After the full deliberation, what is your FINAL VERDICT?
Consider the arguments made during the discussion and explain how they influenced your decision.
Answer only: "GUILTY" or "NOT GUILTY" and give one sentence explaining why.

Format: VERDICT: [GUILTY/NOT GUILTY] - [brief reason referencing the deliberation]"""

        # Add rate limiting between verdict requests
        max_retries = 3
        base_delay = 4
        
        for attempt in range(max_retries):
            try:
                # Progressive delay to avoid rate limits
                if attempt > 0:
                    delay = base_delay * (2 ** attempt) + random.uniform(0, 2)
                    print(f"⏳ Rate limit hit collecting verdict from {jury_name}, waiting {delay:.1f} seconds...")
                    time.sleep(delay)
                else:
                    # Delay between jurors to avoid rapid requests
                    if i > 0:  # No delay for first juror
                        time.sleep(random.uniform(2, 4))
                
                response = llm.invoke([HumanMessage(content=prompt)])
                verdict_line = response.content.strip()
                break
                
            except Exception as e:
                error_msg = str(e)
                if "429" in error_msg or "quota" in error_msg.lower():
                    if attempt < max_retries - 1:
                        continue  # Retry with longer delay
                    else:
                        print(f"❌ Rate limit exceeded collecting verdict from {jury_name}")
                        verdict_line = f"VERDICT: NOT GUILTY - Unable to determine due to system limitations"
                else:
                    print(f"Error getting verdict from {jury_name}: {e}")
                    verdict_line = f"VERDICT: NOT GUILTY - Unable to determine due to technical issue"
                break

        verdicts[jury_name] = verdict_line

    # Count votes
    guilty_votes = sum(1 for v in verdicts.values() if "GUILTY" in v.upper() and "NOT GUILTY" not in v.upper())
    not_guilty_votes = len(verdicts) - guilty_votes

    # Final summary
    summary = "=== FINAL VERDICTS ===\n"
    for jury_name, verdict in verdicts.items():
        summary += f"{jury_name}: {verdict}\n"

    summary += f"\nFINAL TALLY: {guilty_votes} Guilty, {not_guilty_votes} Not Guilty\n"

    if guilty_votes > not_guilty_votes:
        summary += "JURY DECISION: GUILTY"
    elif not_guilty_votes > guilty_votes:
        summary += "JURY DECISION: NOT GUILTY"
    else:
        summary += "JURY DECISION: HUNG JURY (TIE)"

    return {
        "messages": [AIMessage(content=summary, name="Final_Verdict")]
    }

In [117]:
def get_deliberation_summary(messages: List[BaseMessage], max_messages: int = 25) -> str:
    """Create a summary of key discussion points from the deliberation"""

    # Filter out moderator announcements and get actual discussion
    discussion_messages = []
    for msg in messages:
        if hasattr(msg, 'name'):
            # # Skip moderator procedural messages
            # if msg.name == "Moderator" and "===" in msg.content:
            #     continue
            # Skip the initial case presentation
            if msg.name == "User":
                continue
            discussion_messages.append(f"{msg.name}: {msg.content}")

    # Take the most recent discussion points
    if len(discussion_messages) > max_messages:
        discussion_messages = discussion_messages[-max_messages:]

    return "\n".join(discussion_messages)

# Rounds

In [118]:
def should_continue(state: JuryState):
    """Enhanced flow control for multi-round deliberations"""
    current_round = state.get("current_round", 0)
    current_juror_index = state.get("current_juror_index", 0)
    total_rounds = state.get("total_rounds", 3)
    jury_order = state.get("jury_order", [])

    # If no jury order set, we're in trouble
    if not jury_order:
        return "final_verdict"

    # If we've completed all rounds, go to final verdict
    if current_round > total_rounds:
        return "final_verdict"

    # If this is the start (round 0), begin first round
    if current_round == 0:
        return "start_round"

    # If we're in a valid round, determine next action
    if current_juror_index < len(jury_order):
        # Next juror should speak
        return jury_order[current_juror_index]
    else:
        # All jurors have spoken in this round, start next round
        return "start_round"


In [119]:
def start_round(state: JuryState):
    """Start a new round of deliberation"""
    current_round = state.get("current_round", 0) + 1
    total_rounds = state.get("total_rounds", 3)
    jury_order = state.get("jury_order", [])

    # If we've completed all rounds, signal for final verdict
    if current_round > total_rounds:
        msg = AIMessage(content="=== COLLECTING FINAL VERDICTS ===", name="Moderator")
        return {
            "messages": [msg],
            "current_round": current_round,
            "current_juror_index": 0
        }

    # Announce the new round
    msg = AIMessage(content=f"=== DELIBERATION ROUND {current_round} ===", name="Moderator")

    # Start new round with first juror
    return {
        "messages": [msg],
        "current_round": current_round,
        "current_juror_index": 0
    }


In [120]:
def set_deliberation_rounds(total_rounds: int):
    """Set the number of deliberation rounds"""
    global current_total_rounds
    current_total_rounds = total_rounds
    print(f"Set deliberation to {total_rounds} rounds")

# Build the graph

In [121]:
def create_jury_graph(yaml_file: str = None, background_files: List[str] = None, total_rounds: int = 3):
    """Create the jury deliberation graph

    Args:
        yaml_file: Path to YAML file with jury member data
        background_files: List of text files (for backward compatibility)
        total_rounds: Number of deliberation rounds before final verdict
    """

    # Load backgrounds - prioritize YAML file if provided
    if yaml_file:
        backgrounds = load_backgrounds_from_yaml(yaml_file)
    elif background_files:
        backgrounds = load_backgrounds_from_files(background_files)
    else:
        backgrounds = JURY_MEMBERS.copy()

    workflow = StateGraph(JuryState)

    # Add moderator, start_round, and final verdict nodes
    workflow.add_node("moderator", moderator)
    workflow.add_node("start_round", start_round)
    workflow.add_node("final_verdict", final_verdict)

    # Add jury members
    for jury_name in backgrounds.keys():
        workflow.add_node(jury_name, create_jury_node(jury_name))

    # Set up flow
    workflow.add_edge(START, "moderator")
    workflow.add_conditional_edges("moderator", should_continue)

    # start_round determines what happens next
    workflow.add_conditional_edges("start_round", should_continue)

    # Each jury member goes back to flow control
    for jury_name in backgrounds.keys():
        workflow.add_conditional_edges(jury_name, should_continue)

    # Final verdict goes to END
    workflow.add_edge("final_verdict", END)

    return workflow.compile(), backgrounds, total_rounds

# Initialize graph


In [122]:
# Initialize graph
graph, jury_backgrounds, default_rounds = create_jury_graph()

# Stream graph updates

In [123]:
def stream_graph_updates(case_input: str = None, save_to_file: bool = True):
    """Stream jury deliberation updates and optionally save to markdown file"""
    global deliberation_output, current_case_filename, current_scenario_number

    if graph is None:
        print("Cannot run deliberation - API key not configured")
        return

    # Use provided case or current loaded case
    if case_input is None:
        if current_case is None:
            print("No case provided and no case loaded from file")
            return
        case_input = current_case
    else:
        # If case is provided directly, reset file tracking
        if case_input != current_case:
            current_case_filename = None
            current_scenario_number = None

    # Clear previous output and prepare for new deliberation
    deliberation_output = []

    # Create jury order from backgrounds
    jury_order = list(jury_backgrounds.keys())
    juror_colors = assign_juror_colors(jury_order)

    initial_state = {
        "messages": [HumanMessage(content=case_input)],
        "case_details": case_input,
        "jury_backgrounds": jury_backgrounds,
        "current_round": 0,
        "current_juror_index": 0,
        "total_rounds": current_total_rounds,
        "jury_order": jury_order
    }

    # Process the deliberation
    for event in graph.stream(initial_state):
        for value in event.values():
            if "messages" in value and value["messages"]:
                last_message = value["messages"][-1]
                speaker = getattr(last_message, 'name', 'System')
                content = last_message.content

                # Print to console
                print(f"{speaker}: {content}")
                print()

                # Format and store for markdown file
                if save_to_file:
                    formatted_output = format_speaker_output(speaker, content, juror_colors)
                    deliberation_output.append(formatted_output)

    # Save to markdown file if requested
    if save_to_file and deliberation_output:
        save_deliberation_to_markdown(case_input, deliberation_output)

# Save output

In [124]:
# Global variables for output capturing and file tracking
current_case = None
current_total_rounds = default_rounds
deliberation_output = []  # Store all output for saving to file
current_jury_filename = None
current_case_filename = None
current_scenario_number = None

# Create temporary download directory (similar to upload directory setup)
DOWNLOAD_DIR = tempfile.mkdtemp(prefix="jury_downloads_")
print(f"Download directory: {DOWNLOAD_DIR}")

# Color mapping for different speakers
SPEAKER_COLORS = {
    "Moderator": "#2E8B57",      # Sea Green
    "Final_Verdict": "#8B0000",   # Dark Red
    # Default juror colors (will be assigned dynamically)
    "default_colors": [
        "#4169E1",  # Royal Blue
        "#DC143C",  # Crimson
        "#FF8C00",  # Dark Orange
        "#9932CC",  # Dark Orchid
        "#228B22",  # Forest Green
        "#FF1493",  # Deep Pink
        "#8B4513",  # Saddle Brown
        "#00CED1",  # Dark Turquoise
    ]
}

Download directory: C:\Users\user\AppData\Local\Temp\jury_downloads_ah8i6isg


In [125]:
def assign_juror_colors(jury_names):
    """Assign colors to jury members"""
    colors = {}
    available_colors = SPEAKER_COLORS["default_colors"]

    for i, name in enumerate(jury_names):
        if i < len(available_colors):
            colors[name] = available_colors[i]
        else:
            # If more jurors than colors, cycle through
            colors[name] = available_colors[i % len(available_colors)]

    return colors

def format_speaker_output(speaker, content, juror_colors):
    """Format speaker output with colors for markdown"""
    if speaker in juror_colors:
        color = juror_colors[speaker]
    elif speaker in SPEAKER_COLORS:
        color = SPEAKER_COLORS[speaker]
    else:
        color = "#000000"  # Default black

    return f'<span style="color: {color}"><strong>{speaker}:</strong></span> {content}'

def clean_filename_for_output(filepath):
    """Extract clean filename without extension and path"""
    if filepath is None:
        return "unknown"

    # Get just the filename without path
    filename = filepath.split('/')[-1].split('\\')[-1]

    # Remove extension
    filename = filename.rsplit('.', 1)[0]

    # Replace spaces and special characters with underscores
    filename = filename.replace(' ', '_').replace('-', '_')

    # Remove any non-alphanumeric characters except underscores
    filename = re.sub(r'[^a-zA-Z0-9_]', '', filename)

    return filename

def save_deliberation_to_markdown(case_details, output_list, filename=None):
    """Save deliberation output to a markdown file with enhanced naming"""
    if filename is None:
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        # Build filename components
        jury_part = clean_filename_for_output(current_jury_filename) if current_jury_filename else "default_jury"
        case_part = clean_filename_for_output(current_case_filename) if current_case_filename else "direct_case"
        scenario_part = f"scenario{current_scenario_number}" if current_scenario_number else "full"

        filename = f"deliberation_{jury_part}_{case_part}_{scenario_part}_{timestamp}.md"

    # Save to download directory instead of current directory
    full_filepath = os.path.join(DOWNLOAD_DIR, filename)

    # Create markdown content
    markdown_content = f"""# Jury Deliberation Report

**Generated on:** {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}

**Configuration:**
- Jury File: {current_jury_filename or "Default jury members"}
- Case File: {current_case_filename or "Direct input"}
- Scenario: {current_scenario_number if current_scenario_number else "Full case"}
- Rounds: {current_total_rounds}

## Case Details

{case_details}

---

## Deliberation Process

"""

    # Add all the captured output
    for line in output_list:
        markdown_content += line + "\n\n"

    # Add color legend
    markdown_content += "\n---\n\n## Color Legend\n\n"

    # Get current juror colors
    jury_names = list(jury_backgrounds.keys()) if jury_backgrounds else []
    juror_colors = assign_juror_colors(jury_names)

    # Add Moderator and Final Verdict to legend
    all_colors = {**juror_colors, **SPEAKER_COLORS}

    for speaker, color in all_colors.items():
        if speaker != "default_colors":
            markdown_content += f'<span style="color: {color}"><strong>{speaker}</strong></span>\n\n'

    # Save to file
    try:
        with open(full_filepath, 'w', encoding='utf-8') as f:
            f.write(markdown_content)
        print(f"📄 Deliberation saved to: {full_filepath}")
        return full_filepath
    except Exception as e:
        print(f"❌ Error saving deliberation: {e}")
        return None

def get_download_directory():
    """Get the download directory path for external access (e.g., Flask app)"""
    return DOWNLOAD_DIR

def list_download_files():
    """List all files available in the download directory"""
    try:
        if os.path.exists(DOWNLOAD_DIR):
            files = [f for f in os.listdir(DOWNLOAD_DIR) if f.endswith('.md')]
            return [(f, os.path.join(DOWNLOAD_DIR, f)) for f in sorted(files, reverse=True)]
        return []
    except Exception as e:
        print(f"❌ Error listing download files: {e}")
        return []


# Main

## Main interaction loop

In [126]:
# Main interaction loop
def main(interactive: bool = False, save_to_file: bool = True):
    """Main function to run jury deliberation

    Args:
        interactive: If True, runs interactive mode. If False, auto-runs with loaded case.
        save_to_file: Whether to save deliberation to markdown file
    """
    print("=== JURY DELIBERATION SIMULATION ===")

    # Check if LLM is properly initialized
    if llm is None:
        print("\n❌ SETUP REQUIRED:")
        print("1. Get a free API key from: https://aistudio.google.com/app/apikey")
        print("2. Set the environment variable:")
        print("   export GOOGLE_API_KEY='your-api-key-here'")
        print("   (or set it in your script/notebook)")
        print("\nExiting...")
        return

    # If case is pre-loaded and not in interactive mode, run automatically
    if not interactive and current_case is not None:
        print("🚀 Auto-starting deliberation with pre-loaded case and jury...")
        print(f"\nJury Members: {list(jury_backgrounds.keys())}")
        print(f"Deliberation Rounds: {current_total_rounds}")
        print(f"\nCase: {current_case}\n")
        print("🏛️ Starting deliberation...\n")
        stream_graph_updates(save_to_file=save_to_file)
        print("\n🏁 Deliberation completed!")
        return

    # # Interactive mode
    # if interactive:
    #   print("Commands:")
    #   print("• 'load <yaml_file>' - Load jury members from YAML file")
    #   print("• 'load <yaml_file> <rounds>' - Load jury members and set rounds")
    #   print("• 'rounds <number>' - Set number of deliberation rounds")
    #   print("• 'case <case_file>' - Load case from text file")
    #   print("• 'case <case_file> <scenario_number>' - Load specific scenario from file")
    #   print("• 'scenarios <case_file>' - List available scenarios in file")
    #   print("• 'deliberate' - Start deliberation with loaded case")
    #   print("• 'deliberate nosave' - Start deliberation without saving to file")
    #   print("• 'quit', 'exit', or 'q' - Stop")
    #   print("• Or type case details directly for immediate deliberation\n")

    while True:
        try:
            user_input = input("Enter command or case details: ")
            if user_input.lower() in ["quit", "exit", "q"]:
                print("Goodbye!")
                break

            # Check if user wants to load YAML file
            if user_input.lower().startswith("load "):
                parts = user_input[5:].strip().split()
                yaml_file = parts[0]
                rounds = int(parts[1]) if len(parts) > 1 else 3

                try:
                    initialize_with_yaml(yaml_file, rounds)
                    print("Jury members loaded successfully!")
                except Exception as e:
                    print(f"Error loading YAML file: {e}")
                continue

            # Check if user wants to set rounds
            if user_input.lower().startswith("rounds "):
                try:
                    rounds = int(user_input[7:].strip())
                    set_deliberation_rounds(rounds)
                except ValueError:
                    print("Invalid number of rounds. Please enter a number.")
                continue

            # Check if user wants to load case file
            if user_input.lower().startswith("case "):
                parts = user_input[5:].strip().split()
                case_file = parts[0]
                scenario_num = int(parts[1]) if len(parts) > 1 else None

                try:
                    initialize_with_case(case_file, scenario_num)
                    print("Case loaded successfully!")
                except Exception as e:
                    print(f"Error loading case file: {e}")
                continue

            # Check if user wants to list scenarios
            if user_input.lower().startswith("scenarios "):
                case_file = user_input[10:].strip()
                scenarios = list_scenarios_in_file(case_file)
                if scenarios:
                    print(f"Available scenarios in {case_file}:")
                    for num, title in scenarios:
                        print(f"  {num}: {title}")
                else:
                    print("No scenarios found or file error")
                continue

            # Check if user wants to deliberate with loaded case
            if user_input.lower().startswith("deliberate"):
                if current_case is None:
                    print("No case loaded. Use 'case <filename>' to load a case first.")
                    continue

                # Check if user wants to save or not
                save_file = "nosave" not in user_input.lower()

                print(f"\n🏛️ Starting deliberation with loaded case...\n")
                stream_graph_updates(save_to_file=save_file)
                print("\n" + "="*50 + "\n")
                continue

            # Treat as direct case input for immediate deliberation
            if user_input.strip():
                print(f"\nCase: {user_input}\n")
                stream_graph_updates(user_input, save_to_file=save_to_file)
                print("\n" + "="*50 + "\n")

        except KeyboardInterrupt:
            print("\nGoodbye!")
            break
        except Exception as e:
            print(f"An error occurred: {e}")
            print("Trying with fallback example...")
            # Fallback example
            user_input = "John stole a laptop worth $1200 from a coffee shop. He was caught with it 3 days later but claims he bought it from someone on the street for $300. He has no prior record but recently lost his job."
            print("Case: " + user_input)
            stream_graph_updates(user_input, save_to_file=save_to_file)
            break

## `run_deliberation` function

In [127]:
def run_deliberation(jury_file: str = None, case_file: str = None, scenario_number: int = None, total_rounds: int = 3, save_to_file: bool = True):
    """Convenience function to run a complete deliberation session

    Args:
        jury_file: Path to YAML file with jury members
        case_file: Path to text file with case details
        scenario_number: If case file has multiple scenarios, specify which one
        total_rounds: Number of deliberation rounds before final verdict
        save_to_file: Whether to save deliberation to markdown file
    """
    print("=== AUTOMATED JURY DELIBERATION ===")

    if llm is None:
        print("❌ API key not configured. Cannot run deliberation.")
        return

    # Load jury members if specified
    if jury_file:
        try:
            initialize_with_yaml(jury_file, total_rounds)
        except Exception as e:
            print(f"❌ Error loading jury file: {e}")
            return
    else:
        # Set rounds even if no jury file specified
        set_deliberation_rounds(total_rounds)

    # Load case if specified
    if case_file:
        try:
            initialize_with_case(case_file, scenario_number)
        except Exception as e:
            print(f"❌ Error loading case file: {e}")
            return

    # Run deliberation
    if current_case is None:
        print("❌ No case loaded. Cannot start deliberation.")
        return

    print("🚀 Starting automated deliberation...")
    print(f"\nJury Members: {list(jury_backgrounds.keys())}")
    print(f"Deliberation Rounds: {current_total_rounds}")
    print(f"\nCase Preview: {current_case[:200]}{'...' if len(current_case) > 200 else ''}\n")
    print("🏛️ Beginning deliberation...\n")

    stream_graph_updates(save_to_file=save_to_file)
    print("\n🏁 Deliberation completed!")


# Run

## Run `main`

In [128]:
# if __name__ == "__main__":
#
#     initialize_with_yaml("jurors/jurors.yaml", total_rounds=3)
#     initialize_with_case(case_file_path="cases/Scenario 1.txt", scenario_number=2)
#
#     # Display graph visualization
#     # try:
#     #     print("Graph structure:")
#     #     display(Image(graph.get_graph().draw_mermaid_png()))
#     # except Exception as e:
#     #     print(f"Could not display graph: {e}")
#
#     print("\n")
#     main(interactive=False, save_to_file=False)  # Auto-run with pre-loaded case

Loaded jury members from jurors/jurors.yaml:
  - Casey Brown
  - Mitchell Johnson
  - Heather Perez
Set to 3 deliberation rounds

✅ Loaded Scenario 2 from cases/Scenario 1.txt
Case Preview: Scenario 2: 
Background: Tomer and Stan are accused of murder. The prosecution claims they were the only ones seen in the store shortly before the murder and fleeing the scene afterward.
Presenting th...



=== JURY DELIBERATION SIMULATION ===
🚀 Auto-starting deliberation with pre-loaded case and jury...

Jury Members: ['Casey Brown', 'Mitchell Johnson', 'Heather Perez']
Deliberation Rounds: 3

Case: Scenario 2: 
Background: Tomer and Stan are accused of murder. The prosecution claims they were the only ones seen in the store shortly before the murder and fleeing the scene afterward.
Presenting the Decisive Evidence: The trial is underway. The prosecution has presented the Sheriff's testimony confirming that Tomer and Stan were identified at the scene and fled, along with eyewitness testimonies. The

## Run `run_deliberation`

In [129]:
# Commented out - this was interfering with Flask app calls
# run_deliberation(jury_file='republican_and_democratic.yaml',
#                      case_file="Scenario 1.txt",
#                      scenario_number=1,
#                      total_rounds=3,
#                      save_to_file=True)

# All test simulations

In [130]:
# jurors_files = [file for file in  os.listdir() if file.endswith('.yaml')]
# print(jurors_files)

In [131]:
# for jurors_yaml_file in jurors_files:
#   for scenarion_num in range(1, 4):

#     print(f"=== {jurors_yaml_file} ===")

#     # initialize_with_yaml(jurors_yaml_file, total_rounds=3)
#     # initialize_with_case(case_file_path="Scenario 1.txt", scenario_number=scenarion_num)
#     # print("\n")
#     # main(interactive=False, save_to_file=True)  # Auto-run with pre-loaded case

#     run_deliberation(jury_file=jurors_yaml_file,
#                      case_file="Scenario 1.txt",
#                      scenario_number=scenarion_num,
#                      total_rounds=3,
#                      save_to_file=True)